In [4]:
import pandas as pd

In [5]:
### custome functions 
def modify_race_col(row) -> int:
    '''
    logic to make a compariable filed to HISPALLP_C in the post 2019 datasets
    '''
    HISPAN_I = row['HISPAN_I']
    RACERPI2 = row['RACERPI2']
    
    if HISPAN_I != 12:
        return 1

    match RACERPI2:
        case 1:
            return 2
        case 2:
            return 3
        case 3:
            return 5
        case 5:
            return 99
        case 6:
            return 7
        case _:
            return 0

def create_data_file_layout_dict() -> dict:
    '''
    intake text file that filed layouts for .dat 
    files
    '''
    layout_dict = dict()

    with open('data/file_layouts.txt') as f:
        for line in f:
            clean_line = line.replace('\n','').replace('\t','').split(',')
            layout_dict[int(clean_line[0])] = [int(x) for x in clean_line[1:]]

    return layout_dict


def create_dat_file_vars() -> dict:
    #get the field layout for each file
    layout_dict = create_data_file_layout_dict()

    #the fields we will select and for what year
    dat_fields = [1,2,5,6,8,9,12,13,14,17]
    data_2011_2014_fields = [29]
    data_2008_2010_fields = [28]
    data_2005_2007_fields = [25]

    #the renames for each file
    dat_file_renames = {
        1: 'SRVY_YR',
        2: 'HHX',
        5: 'FMX',
        6: 'FPX',
        8: 'WTFA_SC',
        9: 'REGION',
        12: 'SEX',
        13: 'HISPAN_I',
        14: 'RACERPI2',
        17: 'AGE_P',
    }
    dat_file_2011_2014_renames = {29: 'ADD2'}
    dat_file_2008_2010_renames = {28: 'ADD2'}
    dat_file_2005_2007_renames = {25: 'ADD2'}
    
    
    #create final dic used to process the .dat files
    dat_files = {
        2014:{
            'file_nm':'data/child14.dat',
            'file_layout':layout_dict[2014],
            'select_fields':dat_fields + data_2011_2014_fields,
            'rename_fiedls': dat_file_renames | dat_file_2011_2014_renames
        },
        2013:{
            'file_nm':'data/child13.dat',
            'file_layout':layout_dict[2013],
            'select_fields': dat_fields + data_2011_2014_fields,
            'rename_fiedls': dat_file_renames | dat_file_2011_2014_renames  
        },
        2012:{
            'file_nm':'data/child12.dat',
            'file_layout':layout_dict[2012],
            'select_fields': dat_fields + data_2011_2014_fields,
            'rename_fiedls': dat_file_renames | dat_file_2011_2014_renames  
        },
        2011:{
            'file_nm':'data/child11.dat',
            'file_layout':layout_dict[2011],
            'select_fields': dat_fields + data_2011_2014_fields,
            'rename_fiedls': dat_file_renames | dat_file_2011_2014_renames  
        },
        2010:{
            'file_nm':'data/child10.dat',
            'file_layout':layout_dict[2010],
            'select_fields': dat_fields + data_2008_2010_fields,
            'rename_fiedls': dat_file_renames | dat_file_2008_2010_renames  
        },  
        2009:{
            'file_nm':'data/child09.dat',
            'file_layout':layout_dict[2009],
            'select_fields': dat_fields + data_2008_2010_fields,
            'rename_fiedls': dat_file_renames | dat_file_2008_2010_renames  
        },  
        2008:{
            'file_nm':'data/child08.dat',
            'file_layout':layout_dict[2008],
            'select_fields': dat_fields + data_2008_2010_fields,
            'rename_fiedls': dat_file_renames | dat_file_2008_2010_renames  
        },  
        2007:{
            'file_nm':'data/child07.dat',
            'file_layout':layout_dict[2007],
            'select_fields': dat_fields + data_2005_2007_fields,
            'rename_fiedls': dat_file_renames | dat_file_2005_2007_renames  
        },  
        2006:{
            'file_nm':'data/child06.dat',
            'file_layout':layout_dict[2006],
            'select_fields': dat_fields + data_2005_2007_fields,
            'rename_fiedls': dat_file_renames | dat_file_2005_2007_renames  
        }, 
        2005:{
            'file_nm':'data/child05.dat',
            'file_layout':layout_dict[2005],
            'select_fields': dat_fields + data_2005_2007_fields,
            'rename_fiedls': dat_file_renames | dat_file_2005_2007_renames  
        },  
    } 

    return dat_files

In [6]:
## create file list for processing
post_19_files = ('data/child23.csv','data/child22.csv','data/child21.csv','data/child20.csv','data/child19.csv',)

#pre_19 import vars
pre_19_csv = ('data/child18.csv','data/child17.csv','data/child16.csv','data/child15.csv',)
pre_19_renames = {
    'WTFA_SC':'WTFA_C',
    'SEX':'SEX_C',
    'AGE_P':'AGEP_C',
    'ADD2':'ADHDEV_C',
    'ADD2N':'ADHDNW_C',
}


dat_files_2005_2014 = create_dat_file_vars()





In [7]:
### load 2005 to 2014 data
dat_df = pd.DataFrame()

for key, value in dat_files_2005_2014.items():
    df = pd.read_fwf(value['file_nm'],widths=value['file_layout'],header=None)
    df = df[value['select_fields']]
    df = df.rename(columns=value['rename_fiedls'])
    dat_df = pd.concat([dat_df,df])


dat_df.set_index(['HHX','SRVY_YR','FMX','FPX'],inplace=True)
print(dat_df.head())
print(len(dat_df))

                     WTFA_SC  REGION  SEX  HISPAN_I  RACERPI2  AGE_P  ADD2
HHX SRVY_YR FMX FPX                                                       
13  2014    1   3       6039       4    1         3         1      0   NaN
20  2014    1   4       3145       4    2        12         1      2   2.0
25  2014    1   4       5497       2    2        12         1     16   2.0
29  2014    1   5        592       2    2        12         1      7   2.0
34  2014    1   2       6553       1    2        12         2     12   2.0
115390


In [8]:
### load 2015 to 2018
df_2015_2018 = pd.DataFrame()

for file in pre_19_csv:

    if file == 'data/child15.csv':
        cols = ['SRVY_YR','HHX','FMX','FPX','WTFA_SC','REGION','SEX',
                'AGE_P','HISPAN_I','RACERPI2','ADD2',
                ]
    else:
        cols = ['SRVY_YR','HHX','FMX','FPX','WTFA_SC','REGION','SEX',
                'AGE_P','HISPAN_I','RACERPI2','ADD2','ADD2N'
                ]
    
    df = pd.read_csv(file,
                     sep=',',
                     header=0,
                     index_col = ['HHX','SRVY_YR','FMX','FPX'],
                     usecols=cols
                    )

    df_2015_2018 = pd.concat([df_2015_2018,df])

##print the length
print(len(df_2015_2018))



40512


In [19]:
### merge 2015 to 2018 and 2005 to 2014 dataframes and strt to do clean up
df_2005_2018 = pd.concat([dat_df,df_2015_2018]).reset_index()
df_2005_2018.set_index(['HHX','SRVY_YR','FMX','FPX'], inplace=True)


duplicates = df_2005_2018.index.duplicated()
print(df_2005_2018.index[duplicates])
print(len(df_2005_2018))

MultiIndex([], names=['HHX', 'SRVY_YR', 'FMX', 'FPX'])
155902


In [20]:
##create HISPALLP_C
df_2005_2018['HISPALLP_C'] = df_2005_2018.apply(modify_race_col, axis=1)
print(df_2005_2018['HISPALLP_C'].isnull().sum())

##drop columns 'HISPAN_I','RACERPI2' as they are not needed.
df_2005_2018.drop(['HISPAN_I','RACERPI2'],axis=1,inplace=True)


##renaming columns to help combined
df_2005_2018 = df_2005_2018.rename(columns=pre_19_renames)
print(df_2005_2018.head())
print(len(df_2005_2018))

0
                     WTFA_C  REGION  SEX_C  AGEP_C  ADHDEV_C  ADHDNW_C  \
HHX SRVY_YR FMX FPX                                                      
13  2014    1   3      6039       4      1       0       NaN       NaN   
20  2014    1   4      3145       4      2       2       2.0       NaN   
25  2014    1   4      5497       2      2      16       2.0       NaN   
29  2014    1   5       592       2      2       7       2.0       NaN   
34  2014    1   2      6553       1      2      12       2.0       NaN   

                     HISPALLP_C  
HHX SRVY_YR FMX FPX              
13  2014    1   3             1  
20  2014    1   4             2  
25  2014    1   4             2  
29  2014    1   5             2  
34  2014    1   2             3  


In [24]:
## create the final df with the post 2019 files
final_df = pd.DataFrame()

for file in post_19_files:
    df = pd.read_csv(file,
                     sep=',',
                     header=0,
                     index_col = ['HHX','SRVY_YR'],
                     usecols = ['HHX','SRVY_YR','WTFA_C','REGION',
                                'SEX_C','AGEP_C','HISPALLP_C',
                                'ADHDEV_C','ADHDNW_C','LASTDR_C',
                                'RX12M_C','MHRX_C',
                               ]
                    )
    final_df = pd.concat([final_df,df])



final_df['FMX'] = 1
final_df['FPX'] = 1

final_df.reset_index(inplace=True)
final_df.set_index(['HHX','SRVY_YR','FMX','FPX'],inplace=True)

duplicates = final_df.index.duplicated()


print(final_df.head())
print(len(final_df))
print(final_df.index[duplicates])


                         HISPALLP_C  REGION  SEX_C  AGEP_C  MHRX_C  RX12M_C  \
HHX     SRVY_YR FMX FPX                                                       
H045277 2023    1   1             3       3      2      14     2.0        2   
H021192 2023    1   1             2       3      2      11     2.0        2   
H025576 2023    1   1             2       3      1      15     2.0        1   
H058458 2023    1   1             2       3      2       8     2.0        1   
H047432 2023    1   1             3       3      1      12     1.0        1   

                         LASTDR_C  ADHDNW_C  ADHDEV_C     WTFA_C  
HHX     SRVY_YR FMX FPX                                           
H045277 2023    1   1           1       NaN       2.0  13012.875  
H021192 2023    1   1           1       NaN       2.0  16680.509  
H025576 2023    1   1           1       NaN       2.0   5404.923  
H058458 2023    1   1           1       NaN       2.0   9758.143  
H047432 2023    1   1           1       1.0 

In [25]:
final_df = pd.concat([final_df,df_2005_2018]).reset_index()
final_df.set_index(['HHX','SRVY_YR','FMX','FPX'],inplace=True)
print(final_df.head())
print(len(final_df))

                         HISPALLP_C  REGION  SEX_C  AGEP_C  MHRX_C  RX12M_C  \
HHX     SRVY_YR FMX FPX                                                       
H045277 2023    1   1             3       3      2      14     2.0      2.0   
H021192 2023    1   1             2       3      2      11     2.0      2.0   
H025576 2023    1   1             2       3      1      15     2.0      1.0   
H058458 2023    1   1             2       3      2       8     2.0      1.0   
H047432 2023    1   1             3       3      1      12     1.0      1.0   

                         LASTDR_C  ADHDNW_C  ADHDEV_C     WTFA_C  
HHX     SRVY_YR FMX FPX                                           
H045277 2023    1   1         1.0       NaN       2.0  13012.875  
H021192 2023    1   1         1.0       NaN       2.0  16680.509  
H025576 2023    1   1         1.0       NaN       2.0   5404.923  
H058458 2023    1   1         1.0       NaN       2.0   9758.143  
H047432 2023    1   1         1.0       1.0 

In [26]:
## export the file for use latter
final_df.to_csv('data/final_data_set.csv')